# 案例 1.3: 水电站多设备联合调度

本案例演示了水电站的多设备、多目标调度。一个智能体 (`HydropowerStationAgent`) 需要在防洪、供水和发电等多个目标之间取得平衡。

### 1. 导入库并加载配置

In [ ]:
import json
import math
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.hydropower_station_agent import HydropowerStationAgent

CONFIG_PATH = 'examples/hydropower_station_scheduling.json'

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

### 2. 定义仿真函数

该函数将根据配置搭建水电站系统，并模拟一天（24小时）的调度过程，包括一次洪水脉冲事件。

In [ ]:
def run_hydropower_simulation(cfg):
    # 1. 初始化消息总线和仿真参数
    bus = MessageBus()
    sim_cfg = cfg['simulation']
    dt = sim_cfg['dt']

    # 2. 创建水库
    res_cfg = cfg['reservoir']
    initial_volume = res_cfg['initial_volume']
    surface_area = res_cfg['surface_area']
    initial_level = initial_volume / surface_area
    reservoir = Reservoir(
        name="main_reservoir",
        initial_state={'volume': initial_volume, 'water_level': initial_level},
        parameters={'surface_area': surface_area}
    )

    # 3. 创建水闸和水轮机出口
    out_cfg = cfg['outlets']
    shared_params = out_cfg['shared_gate_params']
    outlets = []
    
    tg_cfg = out_cfg['turbine_gate']
    turbine_gates = [Gate(f"turbine_gate_{i+1}", {'opening': 0}, {**shared_params, 'width': tg_cfg['width']}, bus, f"gates/tg{i+1}") for i in range(tg_cfg['num_gates'])]
    outlets.extend(turbine_gates)
    
    fg_cfg = out_cfg['flood_gate']
    flood_gate = Gate("flood_gate", {'opening': 0}, {**shared_params, 'width': fg_cfg['width']}, bus, "gates/flood")
    outlets.append(flood_gate)

    sg_cfg = out_cfg['supply_gate']
    supply_gate = Gate("supply_gate", {'opening': 0}, {**shared_params, 'width': sg_cfg['width']}, bus, "gates/supply")
    outlets.append(supply_gate)
    
    # 4. 创建调度智能体
    agent = HydropowerStationAgent(
        agent_id="HydropowerMaster",
        message_bus=bus,
        reservoir=reservoir,
        flood_gate_topic="gates/flood",
        supply_gate_topic="gates/supply",
        turbine_gate_topics=[f"gates/tg{i+1}" for i in range(tg_cfg['num_gates'])],
        config=cfg['agent_config']
    )

    # 5. 仿真循环
    print("\n--- 开始仿真 ---")
    dist_cfg = cfg['disturbance']
    base_inflow = dist_cfg['base_inflow']
    pulse_cfg = dist_cfg['flood_pulse']
    
    for step in range(sim_cfg['num_steps']):
        # --- 计算扰动 (来水) ---
        inflow = base_inflow
        if pulse_cfg['start_hour'] <= step < pulse_cfg['start_hour'] + pulse_cfg['duration_hours']:
            inflow += pulse_cfg['magnitude'] * math.sin((step - pulse_cfg['start_hour']) / pulse_cfg['duration_hours'] * math.pi)

        print(f"\n--- 小时 {step}, 来水: {inflow:.2f} m^3/s ---")

        # --- 智能体决策 ---
        agent.execute_control_logic()

        # --- 系统物理状态更新 ---
        total_outflow = 0
        reservoir_level = reservoir.get_state()['water_level']
        for outlet in outlets:
            outlet_action = {'upstream_head': reservoir_level, 'downstream_head': 0} # 假设自由出流
            outlet.step(outlet_action, dt)
            total_outflow += outlet.get_state()['outflow']

        reservoir.set_inflow(inflow)
        reservoir.step({'outflow': total_outflow}, dt)

        # --- 日志记录 ---
        print(f"水库水位: {reservoir.get_state()['water_level']:.2f} m")
        for i, tg in enumerate(turbine_gates):
            print(f"  - 发电闸门 {i+1} 开度: {tg.get_state()['opening']:.2f}, 流量: {tg.get_state()['outflow']:.2f}")
        print(f"  - 泄洪闸开度: {flood_gate.get_state()['opening']:.2f}, 流量: {flood_gate.get_state()['outflow']:.2f}")
        print(f"  - 供水闸开度: {supply_gate.get_state()['opening']:.2f}, 流量: {supply_gate.get_state()['outflow']:.2f}")
        print(f"  - 总出库流量: {total_outflow:.2f} m^3/s")

### 3. 运行仿真

In [ ]:
run_hydropower_simulation(config)